In [ ]:
import warnings

warnings.filterwarnings("ignore")
from pathlib import Path
import numpy as np
import arviz as az
from tbdynamics.calibration.az_aux import (
    tabulate_calib_results,
    plot_post_prior_comparison,
    plot_trace,
)
from tbdynamics.camau.calibration.utils import get_bcm
from tbdynamics.camau.constants import params_name
from tbdynamics.calibration.az_aux import (
    process_idata_for_derived_metrics,
    process_priors_for_derived_metrics,
    plot_derived_comparison,
)
from tbdynamics.tools.inputs import get_death_rate, process_universal_death_rate

In [ ]:
RUN_PATH = Path.cwd().parent.parent / 'data/outputs/camau/'
idata = az.from_netcdf(RUN_PATH / 'r0303/calib_full_out.nc')
burnt_idata = idata.sel(draw=np.s_[2500:])

In [ ]:
params = {
    "start_population_size": 30000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
}
covid_effects = {"detection_reduction": True, "contact_reduction": False}
bcm = get_bcm(params, covid_effects, homo_mixing=False)

In [ ]:
bcm.priors

In [ ]:
fig = plot_post_prior_comparison(burnt_idata, bcm.priors, params_name)

In [ ]:
universal_death = process_universal_death_rate(get_death_rate())

In [ ]:
priors = {
    'smear_positive_death_rate': bcm.priors['smear_positive_death_rate'],
    'smear_positive_self_recovery': bcm.priors['smear_positive_self_recovery'],
    'smear_negative_death_rate': bcm.priors['smear_negative_death_rate'],
    'smear_negative_self_recovery': bcm.priors['smear_negative_self_recovery'],
}

In [ ]:
tabulate_calib_results(burnt_idata, params_name)

In [ ]:
tracing = plot_trace(idata, params_name)

In [ ]:
tracing

In [ ]:
# tracing.savefig('../docs/param_traces.png', dpi=300, bbox_inches='tight', format='png', pad_inches=0)

In [ ]:
posterior_metrics = process_idata_for_derived_metrics(burnt_idata, universal_death[2023])

In [ ]:
prior_metrics = process_priors_for_derived_metrics(priors, universal_death[2023])

In [ ]:
plot_derived_comparison(prior_metrics,posterior_metrics);